In [ ]:
import numpy as np
from copy import deepcopy
import random
import time
import os, psutil
import resource

# NOTE: Run Anuj's Experiments.

# Heuristic Functions
There are two heuristics:
- h1 - Heuristic using Manhattan distance
- h2 - Heuristic using number of misplaced tiles

0 represents a blank tile.

In [ ]:
def h1(curr_state, goal_state):
    """
    Heuristic for calculating the distance of goal state using Manhattan distance
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    
    Returns:
    h(int): Heuristic value
    """
    # Indices of that would sort the goal state
    goal_indices = np.argsort(goal_state.reshape(-1,1), axis=0)
    # Indices of that would sort the current state
    curr_indices = np.argsort(curr_state.reshape(-1,1), axis=0)
    # Integer division of the indices by 3 gives the value of first row i.e. indices 0, 1 and 2
    # as 0, second row i.e. indices 3, 4 and 5 as 2 and third row i.e. indices 6, 7 and 8 as 2. 
    x = (abs(goal_indices // 3 - curr_indices // 3))
    # Taking Remainder by 3 gives the value of first column, i.e. indices 0, 3 and 6 as 0 and so on... 
    y = (abs(goal_indices % 3 - curr_indices % 3))
    h = np.sum(x + y)
    return h

In [ ]:
def h2(curr_state, goal_state):
    """
    Heuristic for calculating the distance of goal state using number of misplaced tiles
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    
    Returns:
    h(int): Heuristic value
    """
    # Sums up all the true values returned for different cells in the matrix
    h = np.sum(curr_state != goal_state)
    return h

# Generating random instance of the puzzle at depth d form the goal state

In [ ]:
def generate_instance(goal_state, depth, debug=False):
    """
    Generates a random instance of the 8-puzzle at the given depth from the goal state
    
    Parameters:
    goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements representing the goal state
    depth(int): The depth at which the state is to be generated
    debug(bool): To print intermediate states and the heuristic values, or not. Default: False
    
    Returns:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements representing the state at the given depth form, the goal state
    """
    # Co-ordinates of blank tile of previous state 
    x_prev, y_prev = np.array(np.where(goal_state == 0)).reshape(-1)
    curr_state = np.copy(goal_state)
    # Visited numpy array of flattened state vectors
    visited = np.array([curr_state.reshape(-1)])
    n = depth
    if debug == True:
        print(f"Depth: {depth-n}")
        print(f"h1 value: {h1(curr_state, goal_state)}\th2 value : {h2(curr_state, goal_state)}")
        print(curr_state, end='\n\n')
    while(n):
        # Stores list of all possible locations of blank tile, from the current state
        possible_states = []
        if x_prev > 0:
            possible_states.append([x_prev-1, y_prev])
        if x_prev < 2:
            possible_states.append([x_prev+1, y_prev])
        if y_prev > 0:
            possible_states.append([x_prev, y_prev-1])
        if y_prev < 2:
            possible_states.append([x_prev, y_prev+1])
        # Randomly chooses a position for the blank tile, from the possible choices 
        x_new, y_new = random.choice(possible_states)
        # Swaps the position of previous blank tile with the chosen blank tile
        curr_state[x_new, y_new], curr_state[x_prev, y_prev] = curr_state[x_prev, y_prev], curr_state[x_new, y_new]
        if curr_state.reshape(-1).tolist() not in visited.tolist():
            # If current state is not visited, add it to visited and reduce depth
            visited = np.vstack((visited, curr_state.reshape(-1)))
            n -= 1
            if debug == True:
                if x_new > x_prev:
                    step = "Down"
                elif x_new < x_prev:
                    step = "Up"
                elif y_new > y_prev:
                    step = "Right"
                else:
                    step = "Left"
                print(f"Depth: {depth-n}\tStep taken to reach: {step}")
                print(f"h1 value: {h1(curr_state, goal_state)}\th2 value : {h2(curr_state, goal_state)}")
                print(curr_state, end='\n\n')
            x_prev, y_prev = x_new, y_new
        else:
            # If visited, revert back to previous state and repeat the same process, untill we get a visited state
            curr_state[x_new, y_new], curr_state[x_prev, y_prev] = curr_state[x_prev, y_prev], curr_state[x_new, y_new]
    return curr_state


# Verifying Heuristics and Instance genration

In [ ]:
GOAL_STATE = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print("Goal State:\n", GOAL_STATE)

Goal State:
 [[1 2 3]
 [8 0 4]
 [7 6 5]]


In [ ]:
# Generating random instance
depth = 16
CURR_STATE = generate_instance(GOAL_STATE, depth, debug=False)
print(CURR_STATE)

[[3 4 5]
 [2 0 6]
 [1 8 7]]


In [ ]:
# Heuristic 1
print("Value from h1:", h1(CURR_STATE, GOAL_STATE))

Value from h1: 16


In [ ]:
# Heuristic 2
print("Value from h2:", h2(CURR_STATE, GOAL_STATE))

Value from h2: 8


# General Experiment

In [ ]:
# def find_pos(matrix,value):
#     if value < 0 or value > 8:
#       raise Exception ("Give the value is out of range")
#     else:
#       # print(matrix, value)
#       return np.array(np.where(matrix == value )).reshape(-1)


In [ ]:
# def swap_tiles(matrix,next_pos):
#     row,col=find_pos(matrix,0)
#     # blank_tile=matrix[row][col]
#     # swap_tile=matrix[next_pos[0]][next_pos[1]]
#     # matrix[row][col]=swap_tile
#     # matrix[next_pos[0]][next_pos[1]]=blank_tile

#     matrix[row, col], matrix[next_pos[0], next_pos[1]] = matrix[next_pos[0], next_pos[1]], matrix[row, col]

#     return matrix


In [ ]:
# def possible_moves(current_state):
#     row,col=find_pos(current_state,0)
#     ans_list=[]
#     if(row>0):
#         ans_list.append([row-1,col])
#     if(row<2):
#         ans_list.append([row+1,col])
#     if(col>0):
#         ans_list.append([row,col-1])
#     if(col<2):
#         ans_list.append([row,col+1])
#     return ans_list

In [ ]:
#  visited = np.array([0]*9)

In [ ]:
# def solve(curr_state,visited):
#     next_possible=[]
#     heurestic_val=[]
#     choosen_matrix=curr_state
#     visited = np.vstack((visited, choosen_matrix.reshape(-1)))
#     if (choosen_matrix == GOAL_STATE).all():
#         print("REACHED THE GOAL STATE")
#         return 
#     for i in possible_moves(curr_state):
#        if np.array(i).reshape(-1).tolist() not in visited.tolist():
#         matrix2=deepcopy(curr_state)
#         temp=swap_tiles(matrix2,i)
#         next_possible.append(temp)

#         c=h1(temp,GOAL_STATE) # calculate herestic val
#         heurestic_val.append(c)
#     for k in range(len(next_possible)):
#         if(heurestic_val[k]==min(heurestic_val)):
#             visited = np.vstack((visited, next_possible[k].reshape(-1)))
#             choosen_matrix=next_possible[k]
#     print(choosen_matrix)
#     # choosen_dict = mat_to_dict(choosen_matrix)
#     if (choosen_matrix == GOAL_STATE).all():
#         print("REACHED THE GOAL STATE")
#         return
#     else:
#         print(visited)
#         solve(choosen_matrix,visited)



In [ ]:
CURR_STATE = generate_instance(GOAL_STATE, 10)

In [ ]:
print(CURR_STATE)

[[2 3 4]
 [1 0 5]
 [8 6 7]]


In [ ]:
# solve(CURR_STATE, visited)
# print(visited)

Streaming output truncated to the last 5000 lines.
[[2 3 4]
 [1 0 5]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 6 7]
 [2 0 4 ... 8 6 7]
 ...
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]]
[[2 3 4]
 [1 5 0]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 6 7]
 [2 0 4 ... 8 6 7]
 ...
 [2 3 4 ... 8 0 7]
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]]
[[2 3 4]
 [1 0 5]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 6 7]
 [2 0 4 ... 8 6 7]
 ...
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]]
[[2 3 4]
 [1 5 0]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 6 7]
 [2 0 4 ... 8 6 7]
 ...
 [2 3 4 ... 8 0 7]
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]]
[[2 3 4]
 [1 0 5]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 6 7]
 [2 0 4 ... 8 6 7]
 ...
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]]
[[2 3 4]
 [1 5 0]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 6 7]
 [2 0 4 ... 8 6 7]
 ...
 [2 3 4 ... 8 0 7]
 [2 3 4 ... 8 6 7]
 [2 3 4 ... 8 6 7]]
[[2 3 4]
 [1 0 5]
 [8 6 7]]
[[0 0 0 ... 0 0 0]
 [2 3 4 ... 8 

RecursionError: ignored

# Pushkar's Experiments

In [ ]:
# def get_possible_moves(curr_state):
#     """
#     Returns a list of possible states after valid moves form current state
    
#     Parameters:
#     curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements, representing the current state
    
#     Returns:
#     possible_moves(list): List of possible states after valid moves form current state
#     """
#     row, col = np.array(np.where(curr_state == 0)).reshape(-1)
#     possible_moves = []
#     if(row > 0):
#         next_state = curr_state.copy()
#         next_state[row, col], next_state[row-1, col] = next_state[row-1, col], next_state[row, col]
#         possible_moves.append(next_state)
#     if(row < 2):
#         next_state = curr_state.copy()
#         next_state[row, col], next_state[row+1, col] = next_state[row+1, col], next_state[row, col]
#         possible_moves.append(next_state)
#     if(col > 0):
#         next_state = curr_state.copy()
#         next_state[row, col], next_state[row, col-1] = next_state[row, col-1], next_state[row, col]
#         possible_moves.append(next_state)
#     if(col < 2):
#         next_state = curr_state.copy()
#         next_state[row, col], next_state[row, col+1] = next_state[row, col+1], next_state[row, col]
#         possible_moves.append(next_state)
#     return possible_moves

In [ ]:
# def sort_by_heuristic(possible_moves, goal_state, heuristic):
#     """
#     Helper function to sort the next possible states based on heuristic value passed
    
#     Parameters:
#     possible_moves(list): List of possible states after valid moves form current state
#     goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements representing the goal state
#     heuristic(Integer): An integer indicating the heuristic function to use. 1 for heuristic h1 and 2 for heurostic h2
    
#     Returns:
#     sorted_possible_moves(list): List of possible states after valid moves form current state, sorted according to heuristic
#     """
#     if heuristic == 1:
#         sorted_possible_moves = sorted(possible_moves, key=lambda x: h1(x, goal_state))
#     if heuristic == 2:
#         sorted_possible_moves = sorted(possible_moves, key=lambda x:h2(x, goal_state))
#     return sorted_possible_moves

In [ ]:
# def solve2(curr_state, goal_state, heuristic=0):
#     """
#     Solves the puzzle by finding a path from current state to the goal state, using the heuristic provided.
#     If no heuristic is provided, solves using normal BFS. Prints "GOAL REACHED!!!" if goal is reached and
#     prints "STRUCK!!!" if no possible move is left

#     Parameters:
#     curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements, representing the current state
#     goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements representing the goal state
#     heuristic(Integer): An integer indicating the heuristic function to use. 1 for heuristic h1 and 2 for heurostic h2. If
#     not provided or passed as 0, solves using nortmal BFS
#     """
#     visited = np.array([])
#     frontier = np.array([curr_state.reshape(-1)])
#     while (curr_state.tolist() != goal_state.tolist()):
#         print(curr_state, end="\n\n")
#         possible_moves = get_possible_moves(curr_state)
#         if heuristic != 0:
#             possible_moves = sort_by_heuristic(possible_moves, goal_state, heuristic)
#         for move in possible_moves:
#             curr_move = move.reshape(-1).tolist()
#             if curr_move not in frontier.tolist():
#                 if curr_move not in visited.tolist():
#                     frontier = np.vstack((frontier, move.reshape(-1)))
#         if (frontier == np.array([])):
#             print("STRUCK!!!")
#             return
#         if (visited == np.array([])).all():
#             visited = np.array(curr_state.reshape(-1,1))
#         else:
#             visited = np.vstack((visited, curr_state.reshape(-1)))
#         frontier = np.delete(frontier, 0, 0)
#         curr_state = frontier[0].reshape(3,3)
#     print("GOAL REACHED!!!")
#     print(curr_state)

# Testing Pushkar's Experiments

In [ ]:
CURR_STATE = generate_instance(GOAL_STATE, 8)
print(CURR_STATE)

[[2 8 3]
 [1 4 5]
 [0 7 6]]


In [ ]:
solve2(CURR_STATE, GOAL_STATE, 1)

[[2 8 3]
 [1 4 5]
 [0 7 6]]

[[2 8 3]
 [1 4 5]
 [7 0 6]]

[[2 8 3]
 [0 4 5]
 [1 7 6]]

[[2 8 3]
 [1 0 5]
 [7 4 6]]

[[2 8 3]
 [1 4 5]
 [7 6 0]]

[[2 8 3]
 [1 4 5]
 [0 7 6]]

[[2 8 3]
 [4 0 5]
 [1 7 6]]

[[0 8 3]
 [2 4 5]
 [1 7 6]]

[[2 0 3]
 [1 8 5]
 [7 4 6]]

[[2 8 3]
 [1 4 5]
 [7 0 6]]

[[2 8 3]
 [0 1 5]
 [7 4 6]]

[[2 8 3]
 [1 5 0]
 [7 4 6]]

[[2 8 3]
 [1 4 0]
 [7 6 5]]

[[2 8 3]
 [0 4 5]
 [1 7 6]]

[[2 0 3]
 [4 8 5]
 [1 7 6]]

[[2 8 3]
 [4 7 5]
 [1 0 6]]

[[2 8 3]
 [4 5 0]
 [1 7 6]]

[[8 0 3]
 [2 4 5]
 [1 7 6]]

[[2 8 3]
 [1 0 5]
 [7 4 6]]

[[0 2 3]
 [1 8 5]
 [7 4 6]]

[[2 3 0]
 [1 8 5]
 [7 4 6]]

[[2 8 3]
 [1 4 5]
 [7 6 0]]

[[2 8 3]
 [1 4 5]
 [0 7 6]]

[[0 8 3]
 [2 1 5]
 [7 4 6]]

[[2 8 3]
 [7 1 5]
 [0 4 6]]

[[2 8 0]
 [1 5 3]
 [7 4 6]]

[[2 8 3]
 [1 5 6]
 [7 4 0]]

[[2 8 3]
 [1 0 4]
 [7 6 5]]

[[2 8 0]
 [1 4 3]
 [7 6 5]]

[[2 8 3]
 [4 0 5]
 [1 7 6]]

[[0 8 3]
 [2 4 5]
 [1 7 6]]

[[0 2 3]
 [4 8 5]
 [1 7 6]]

[[2 3 0]
 [4 8 5]
 [1 7 6]]

[[2 8 3]
 [4 7 5]
 [1 6 0]]

[[2 8 3]
 [4 7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.




[[1 2 3]
 [8 0 5]
 [7 4 6]]

[[1 2 3]
 [7 8 5]
 [0 4 6]]

[[2 3 5]
 [1 0 8]
 [7 4 6]]

[[2 3 5]
 [1 8 6]
 [7 4 0]]

[[2 8 3]
 [1 0 4]
 [7 6 5]]

[[2 8 0]
 [1 4 3]
 [7 6 5]]

[[2 8 3]
 [4 0 5]
 [1 7 6]]

[[0 8 3]
 [2 4 5]
 [1 7 6]]

[[8 1 3]
 [2 0 5]
 [7 4 6]]

[[8 3 0]
 [2 1 5]
 [7 4 6]]

[[2 8 3]
 [7 0 5]
 [4 1 6]]

[[2 8 3]
 [7 1 5]
 [4 6 0]]

[[2 5 8]
 [1 0 3]
 [7 4 6]]

[[0 2 8]
 [1 5 3]
 [7 4 6]]

[[2 8 3]
 [1 0 6]
 [7 5 4]]

[[2 8 3]
 [1 5 6]
 [0 7 4]]

[[1 2 3]
 [8 4 5]
 [7 6 0]]

[[1 2 3]
 [8 4 5]
 [0 7 6]]

[[0 1 3]
 [8 2 5]
 [7 4 6]]

[[1 3 0]
 [8 2 5]
 [7 4 6]]

[[1 2 0]
 [8 5 3]
 [7 4 6]]

[[1 2 3]
 [8 5 6]
 [7 4 0]]

[[1 2 3]
 [7 0 5]
 [4 8 6]]

[[1 2 3]
 [7 8 5]
 [4 6 0]]

[[2 3 5]
 [1 4 8]
 [7 6 0]]

[[2 3 5]
 [1 4 8]
 [0 7 6]]

[[0 2 5]
 [1 3 8]
 [7 4 6]]

[[2 5 0]
 [1 3 8]
 [7 4 6]]

[[0 3 5]
 [2 1 8]
 [7 4 6]]

[[2 3 5]
 [7 1 8]
 [0 4 6]]

[[2 3 5]
 [1 0 6]
 [7 8 4]]

[[2 3 5]
 [1 8 6]
 [0 7 4]]

[[0 2 3]
 [1 8 4]
 [7 6 5]]

[[2 3 0]
 [1 8 4]
 [7 6 5]]

[[2 8 3]
 [1

Works for inputs upto atleast 16

# TODO#1: Test for inputs higher than 16, preferably, test for 32 and 64

In [ ]:
CURR_STATE32 = generate_instance(GOAL_STATE, 32)
print(CURR_STATE32)

[[0 3 7]
 [1 6 8]
 [2 4 5]]


In [ ]:
# Test this
solve2(CURR_STATE32, GOAL_STATE, 1)

[[0 3 7]
 [1 6 8]
 [2 4 5]]

[[1 3 7]
 [0 6 8]
 [2 4 5]]

[[3 0 7]
 [1 6 8]
 [2 4 5]]

[[1 3 7]
 [2 6 8]
 [0 4 5]]

[[1 3 7]
 [6 0 8]
 [2 4 5]]

[[0 3 7]
 [1 6 8]
 [2 4 5]]

[[3 6 7]
 [1 0 8]
 [2 4 5]]

[[3 7 0]
 [1 6 8]
 [2 4 5]]

[[1 3 7]
 [0 6 8]
 [2 4 5]]

[[1 3 7]
 [2 6 8]
 [4 0 5]]

[[1 3 7]
 [6 4 8]
 [2 0 5]]

[[1 3 7]
 [6 8 0]
 [2 4 5]]

[[1 0 7]
 [6 3 8]
 [2 4 5]]

[[3 0 7]
 [1 6 8]
 [2 4 5]]

[[3 6 7]
 [1 4 8]
 [2 0 5]]

[[3 6 7]
 [1 8 0]
 [2 4 5]]

[[3 6 7]
 [0 1 8]
 [2 4 5]]

[[3 7 8]
 [1 6 0]
 [2 4 5]]

[[1 3 7]
 [2 6 8]
 [0 4 5]]

[[1 3 7]
 [6 0 8]
 [2 4 5]]

[[0 3 7]
 [1 6 8]
 [2 4 5]]

[[1 3 7]
 [2 0 8]
 [4 6 5]]

[[1 3 7]
 [2 6 8]
 [4 5 0]]

[[1 3 7]
 [6 4 8]
 [0 2 5]]

[[1 3 7]
 [6 4 8]
 [2 5 0]]

[[1 3 0]
 [6 8 7]
 [2 4 5]]

[[1 3 7]
 [6 8 5]
 [2 4 0]]

[[1 7 0]
 [6 3 8]
 [2 4 5]]

[[0 1 7]
 [6 3 8]
 [2 4 5]]

[[3 6 7]
 [1 0 8]
 [2 4 5]]

[[3 7 0]
 [1 6 8]
 [2 4 5]]

[[3 6 7]
 [1 4 8]
 [0 2 5]]

[[3 6 7]
 [1 4 8]
 [2 5 0]]

[[3 6 0]
 [1 8 7]
 [2 4 5]]

[[3 6 7]
 [1 8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Streaming output truncated to the last 5000 lines.
[[1 3 7]
 [2 6 5]
 [8 4 0]]

[[3 7 0]
 [1 6 5]
 [2 8 4]]

[[1 7 5]
 [6 0 3]
 [2 8 4]]

[[1 7 5]
 [6 3 4]
 [2 8 0]]

[[6 1 7]
 [2 3 5]
 [0 8 4]]

[[6 1 7]
 [3 0 5]
 [2 8 4]]

[[1 5 3]
 [6 0 7]
 [2 8 4]]

[[0 1 3]
 [6 5 7]
 [2 8 4]]

[[1 3 7]
 [8 2 5]
 [0 6 4]]

[[1 3 7]
 [8 2 5]
 [6 4 0]]

[[1 7 0]
 [8 3 5]
 [6 2 4]]

[[0 1 7]
 [8 3 5]
 [6 2 4]]

[[1 3 0]
 [8 5 7]
 [6 2 4]]

[[1 3 7]
 [8 5 4]
 [6 2 0]]

[[3 8 7]
 [1 0 5]
 [6 2 4]]

[[3 7 0]
 [1 8 5]
 [6 2 4]]

[[6 1 8]
 [7 0 3]
 [2 4 5]]

[[6 1 8]
 [2 7 3]
 [0 4 5]]

[[0 7 8]
 [1 4 3]
 [6 2 5]]

[[1 7 8]
 [6 0 4]
 [2 5 3]]

[[1 7 0]
 [6 4 8]
 [2 5 3]]

[[7 6 8]
 [1 0 3]
 [2 4 5]]

[[7 8 0]
 [1 6 3]
 [2 4 5]]

[[1 8 0]
 [6 7 5]
 [2 3 4]]

[[0 1 8]
 [6 7 5]
 [2 3 4]]

[[1 7 8]
 [2 6 5]
 [0 3 4]]

[[0 7 8]
 [1 6 5]
 [2 3 4]]

[[1 7 0]
 [6 5 8]
 [2 3 4]]

[[1 7 8]
 [6 5 4]
 [2 3 0]]

[[1 7 8]
 [3 0 5]
 [6 2 4]]

[[0 7 8]
 [1 3 5]
 [6 2 4]]

[[0 1 7]
 [6 2 8]
 [4 3 5]]

[[6 1 7]
 [4 2 8]
 [0

KeyboardInterrupt: ignored

In [ ]:
CURR_STATE64 = generate_instance(GOAL_STATE, 64)
print(CURR_STATE64)

In [ ]:
# Test this
solve2(CURR_STATE64, GOAL_STATE, 1)

# TODO#2: Maintain the moves made to reach the goal state from the current state, total nodes visited in this process and the size of frontier at that instance and the time required to run for different input sizes, in all both the heuristics and without any heuristics

### Time and memory requirement

In [ ]:
import time
import os, psutil
import resource
import sys

In [ ]:
#mem: https://stackoverflow.com/a/7669482/10505839
#time: https://stackoverflow.com/a/7370824/10505839

sys.setrecursionlimit(10**6) 

print(f"pagesize: {resource.getpagesize()}")
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

pagesize: 4096


325268

In [ ]:
depths = np.array([2,4,8])
print(depths)
for d in depths:
  start = time.perf_counter()
  CURR_STATE = generate_instance(GOAL_STATE, d)
  solve(CURR_STATE, visited)
  end = time.perf_counter()
  print(f"Depth: {d}\tTime Taken: {end-start}\tMax Mem: {resource.getrusage(resource.RUSAGE_SELF).ru_maxrss}")


Streaming output truncated to the last 5000 lines.
[[1 2 3]
 [0 6 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 ...
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]]
[[1 2 3]
 [6 0 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 ...
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 0 7 5]
 [1 2 3 ... 8 7 5]]
[[1 2 3]
 [0 6 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 ...
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]]
[[1 2 3]
 [6 0 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 ...
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 0 7 5]
 [1 2 3 ... 8 7 5]]
[[1 2 3]
 [0 6 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 ...
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]]
[[1 2 3]
 [6 0 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 8 7 5]
 ...
 [1 2 3 ... 8 7 5]
 [1 2 3 ... 0 7 5]
 [1 2 3 ... 8 7 5]]
[[1 2 3]
 [0 6 4]
 [8 7 5]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 8 

# Solving the Puzzle

In [ ]:
def get_possible_moves(curr_state,parent):
    """
    Returns a list of possible states after valid moves form current state
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements, representing the current state
    parent(string): The path taken to reach thr current state from initial Arrangement
    
    Returns:
    possible_moves(list): List of possible states after valid moves form current state
    possible_paths(list): List of possible paths after valid moves form current state
    """
    row, col = np.array(np.where(curr_state == 0)).reshape(-1)
    possible_moves = []
    possible_paths = []

    if(row > 0):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row-1, col] = next_state[row-1, col], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"U->")
    if(row < 2):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row+1, col] = next_state[row+1, col], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"D->")
    if(col > 0):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row, col-1] = next_state[row, col-1], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"L->")
    if(col < 2):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row, col+1] = next_state[row, col+1], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"R->")
    return possible_moves,possible_paths

In [ ]:
def sort_by_heuristic(possible_moves,possible_paths, goal_state, heuristic):
    """
    Helper function to sort the next possible states based on heuristic value passed
    
    Parameters:
    possible_moves(list): List of possible states after valid moves form current state
    possible_paths(list): List of possible moves after valid moves form current state
    goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements representing the goal state
    heuristic(Integer): An integer indicating the heuristic function to use. 1 for heuristic h1 and 2 for heurostic h2
    
    Returns:
    sorted_possible_moves(list): List of possible states after valid moves form current state, sorted according to heuristic
    sorted_possible_moves(list): List of possible paths after valid moves form current state, sorted according to heuristic
    """
    if heuristic == 1:
        # sorted_possible_moves,sorted_possible_paths = sorted(possible_moves, key=lambda x: h1(x, goal_state))
         sorted_possible_moves,sorted_possible_paths = zip(*sorted(zip(possible_moves,possible_paths),key=lambda x: h1(x[0], goal_state)))
    if heuristic == 2:
        # sorted_possible_moves = sorted(possible_moves, key=lambda x:h2(x, goal_state))
         sorted_possible_moves,sorted_possible_paths = zip(*sorted(zip(possible_moves,possible_paths),key=lambda x: h2(x[0], goal_state)))

    return sorted_possible_moves,sorted_possible_paths

In [ ]:
def solve(curr_state, goal_state, heuristic=0):
    """
    Solves the puzzle by finding a path from current state to the goal state, using the heuristic provided.
    If no heuristic is provided, solves using normal BFS. Prints "GOAL REACHED!!!" if goal is reached and
    prints "STRUCK!!!" if no possible move is left

    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements, representing the current state
    goal_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements representing the goal state
    heuristic(Integer): An integer indicating the heuristic function to use. 1 for heuristic h1 and 2 for heurostic h2. If
    not provided or passed as 0, solves using nortmal BFS
    """
    visited = np.array([])
    frontier = np.array([curr_state.reshape(-1)])
    dict={str(curr_state.tolist()):"Start->"}
    while (curr_state.tolist() != goal_state.tolist()):
        print(curr_state, end="\n\n")
        possible_moves,possible_paths = get_possible_moves(curr_state,dict[str(curr_state.tolist())])
        if heuristic != 0:
            possible_moves,possible_paths = sort_by_heuristic(possible_moves, possible_paths,goal_state, heuristic)
        for ind, move in enumerate(possible_moves):
            curr_move = move.reshape(-1).tolist()
            if curr_move not in frontier.tolist():
                if curr_move not in visited.tolist():
                    frontier = np.vstack((frontier, move.reshape(-1)))
                    # move_taken= possible_paths[possible_moves.index(move)]
                    # index=np.where((possible_moves==move).all())
                    move_taken= possible_paths[ind]
                    dict[str(move.tolist())]=move_taken
        if (frontier == np.array([])):
            print("STRUCK!!!")
            returnd
        if (visited == np.array([])).all():
            visited = np.array(curr_state.reshape(-1,1))
        else:
            visited = np.vstack((visited, curr_state.reshape(-1)))
        frontier = np.delete(frontier, 0, 0)
        curr_state = frontier[0].reshape(3,3)
    print("GOAL REACHED!!!")
    print(curr_state)
    print(dict[str(curr_state.tolist())]+"Goal")

NameError: ignored

In [ ]:
CURR_STATE = generate_instance(GOAL_STATE, 7)
print(CURR_STATE)
solve(CURR_STATE, GOAL_STATE, 1)

[[1 3 4]
 [8 6 0]
 [7 5 2]]
[[1 3 4]
 [8 6 0]
 [7 5 2]]

[[1 3 0]
 [8 6 4]
 [7 5 2]]

[[1 3 4]
 [8 6 2]
 [7 5 0]]

[[1 3 4]
 [8 0 6]
 [7 5 2]]

[[1 0 3]
 [8 6 4]
 [7 5 2]]

[[1 3 4]
 [8 6 0]
 [7 5 2]]

[[1 3 4]
 [8 6 2]
 [7 0 5]]

[[1 0 4]
 [8 3 6]
 [7 5 2]]

[[1 3 4]
 [8 5 6]
 [7 0 2]]

[[1 3 4]
 [0 8 6]
 [7 5 2]]

[[1 6 3]
 [8 0 4]
 [7 5 2]]

[[0 1 3]
 [8 6 4]
 [7 5 2]]

[[1 3 0]
 [8 6 4]
 [7 5 2]]

[[1 3 4]
 [8 6 2]
 [7 5 0]]

[[1 3 4]
 [8 0 6]
 [7 5 2]]

[[1 3 4]
 [8 0 2]
 [7 6 5]]

[[1 3 4]
 [8 6 2]
 [0 7 5]]

[[0 1 4]
 [8 3 6]
 [7 5 2]]

[[1 4 0]
 [8 3 6]
 [7 5 2]]

[[1 3 4]
 [8 5 6]
 [7 2 0]]

[[1 3 4]
 [8 5 6]
 [0 7 2]]

[[0 3 4]
 [1 8 6]
 [7 5 2]]

[[1 3 4]
 [7 8 6]
 [0 5 2]]

[[1 0 3]
 [8 6 4]
 [7 5 2]]

[[1 6 3]
 [8 5 4]
 [7 0 2]]

[[1 6 3]
 [0 8 4]
 [7 5 2]]

[[1 6 3]
 [8 4 0]
 [7 5 2]]

[[8 1 3]
 [0 6 4]
 [7 5 2]]

[[1 3 4]
 [8 6 0]
 [7 5 2]]

[[1 3 4]
 [8 6 2]
 [7 0 5]]

[[1 0 4]
 [8 3 6]
 [7 5 2]]

[[1 3 4]
 [8 5 6]
 [7 0 2]]

[[1 3 4]
 [0 8 6]
 [7 5 2]]

[[1 3 4]
 [8 2 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


# Ashutosh's experiments

Memory and Time comparison table

Calculating the total times taken by the program for finding the solution. Using depth 2, 4, 8, 16, 32 and maybe 64.


In [ ]:
#mem: https://stackoverflow.com/a/7669482/10505839
#time: https://stackoverflow.com/a/7370824/10505839

# sys.setrecursionlimit(10**6) 

print(f"pagesize: {resource.getpagesize()}")
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

pagesize: 4096


109036

In [ ]:
depths = np.array([2,4,8])
print(depths)
for d in depths:
  start = time.perf_counter()
  CURR_STATE = generate_instance(GOAL_STATE, d)
  solve2(CURR_STATE, GOAL_STATE, 1)
  end = time.perf_counter()
  print(f"Depth: {d}\tTime Taken: {end-start}\tMax Mem: {resource.getrusage(resource.RUSAGE_SELF).ru_maxrss}")


[2 4 8]
[[1 2 3]
 [8 6 4]
 [7 5 0]]

[[1 2 3]
 [8 6 4]
 [7 0 5]]

[[1 2 3]
 [8 6 0]
 [7 5 4]]

Memory used byted: 122044416
GOAL REACHED!!!
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Start->L->U->Goal
Depth: 2	Time Taken: 0.006281345000388683	Max Mem: 119188
[[0 2 3]
 [1 6 4]
 [8 7 5]]

[[1 2 3]
 [0 6 4]
 [8 7 5]]

[[2 0 3]
 [1 6 4]
 [8 7 5]]

[[1 2 3]
 [8 6 4]
 [0 7 5]]

[[1 2 3]
 [6 0 4]
 [8 7 5]]

[[0 2 3]
 [1 6 4]
 [8 7 5]]

[[2 6 3]
 [1 0 4]
 [8 7 5]]

[[2 3 0]
 [1 6 4]
 [8 7 5]]

[[1 2 3]
 [8 6 4]
 [7 0 5]]

[[1 2 3]
 [0 6 4]
 [8 7 5]]

[[1 0 3]
 [6 2 4]
 [8 7 5]]

[[1 2 3]
 [6 7 4]
 [8 0 5]]

[[1 2 3]
 [6 4 0]
 [8 7 5]]

[[2 0 3]
 [1 6 4]
 [8 7 5]]

[[2 6 3]
 [1 7 4]
 [8 0 5]]

[[2 6 3]
 [0 1 4]
 [8 7 5]]

[[2 6 3]
 [1 4 0]
 [8 7 5]]

[[2 3 4]
 [1 6 0]
 [8 7 5]]

Memory used byted: 122044416
GOAL REACHED!!!
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Start->D->D->R->U->Goal
Depth: 4	Time Taken: 0.023729329999696347	Max Mem: 119188
[[1 8 2]
 [7 0 3]
 [6 5 4]]

[[1 0 2]
 [7 8 3]
 [6 5 4]]

[[1 8 2]
 [7 5 3]
 [6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


[[7 1 2]
 [6 8 3]
 [5 0 4]]

[[1 0 2]
 [5 8 3]
 [7 6 4]]

[[1 8 2]
 [5 6 3]
 [7 0 4]]

[[1 8 2]
 [5 3 0]
 [7 6 4]]

[[8 0 2]
 [1 5 3]
 [7 6 4]]

[[1 0 2]
 [7 8 5]
 [6 4 3]]

[[1 8 2]
 [7 4 5]
 [6 0 3]]

[[1 8 2]
 [0 7 5]
 [6 4 3]]

[[1 0 8]
 [7 5 2]
 [6 4 3]]

[[8 7 2]
 [1 5 3]
 [6 0 4]]

[[8 7 2]
 [0 1 3]
 [6 5 4]]

[[8 7 2]
 [1 3 0]
 [6 5 4]]

[[8 2 3]
 [1 7 0]
 [6 5 4]]

[[1 0 2]
 [6 8 3]
 [5 7 4]]

[[1 8 2]
 [0 6 3]
 [5 7 4]]

[[1 8 2]
 [6 3 0]
 [5 7 4]]

[[1 8 2]
 [6 7 0]
 [5 4 3]]

[[1 0 2]
 [7 8 4]
 [6 3 5]]

[[1 8 2]
 [0 7 4]
 [6 3 5]]

[[1 8 2]
 [7 4 0]
 [6 3 5]]

[[1 8 2]
 [0 3 4]
 [7 6 5]]

[[1 3 8]
 [7 2 0]
 [6 5 4]]

[[1 3 8]
 [7 5 2]
 [6 0 4]]

[[1 3 8]
 [0 7 2]
 [6 5 4]]

[[7 1 8]
 [0 3 2]
 [6 5 4]]

[[1 0 3]
 [7 2 4]
 [6 8 5]]

[[1 2 3]
 [0 7 4]
 [6 8 5]]

[[1 2 3]
 [7 4 0]
 [6 8 5]]

[[1 2 3]
 [0 8 4]
 [7 6 5]]

[[7 1 3]
 [0 2 8]
 [6 5 4]]

[[1 2 3]
 [0 5 8]
 [7 6 4]]

[[1 2 3]
 [7 5 0]
 [6 4 8]]

[[2 0 3]
 [1 7 8]
 [6 5 4]]

[[1 2 3]
 [6 7 8]
 [5 0 4]]

[[7 2 3]
 [8 1

[R, L, U] -> Parent
D -> [R, L, U, D]


16 depth
